In [1]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import anndata
import scvi
import scanpy as sc
import matplotlib
import csv
import os
seed=10
os.chdir(".../objects")
sc.logging.print_versions()

Global seed set to 0


-----
anndata     0.8.0
scanpy      1.9.1
-----
PIL                 9.2.0
absl                NA
asttokens           NA
attr                22.1.0
backcall            0.2.0
bottleneck          1.3.5
cffi                1.15.1
chex                0.1.5
colorama            0.4.5
constants           NA
cycler              0.10.0
cython_runtime      NA
dateutil            2.8.2
debugpy             1.6.3
decorator           4.4.2
defusedxml          0.7.1
deprecate           0.3.2
docrep              0.3.2
entrypoints         0.4
etils               0.8.0
executing           1.0.0
flax                0.6.0
fsspec              2022.8.2
google              NA
h5py                3.7.0
highs_wrapper       NA
igraph              0.9.11
ipykernel           6.15.3
ipython_genutils    0.2.0
ipywidgets          8.0.2
jax                 0.3.17
jaxlib              0.3.15
jedi                0.18.0
joblib              1.2.0
jupyter_server      2.0.4
kiwisolver          1.4.4
leidenalg           0.8.1

In [ ]:
adata = scvi.data.read_h5ad('Atlas6.1_3000HVG_integrated_V2.h5ad')


In [ ]:
model = scvi.model.SCVI.load("/home/kloetzer/Atlas/Models/Model_Atlas6.1_V2", adata=adata, use_gpu=False)

In [5]:
adata.obs['annotation_Atlas_level2']=adata.obs.leiden_scVI_2_0
Rename_cluster = {
"0": 'PTS1_PTS2',
"1": 'PTS1_PTS2',
"2": 'injPT',
"3": 'PTS1_PTS2',
"4": 'DCT',
"5": 'EC3',
"6": 'PTS3',
"7": 'TAL4',
"8": 'PTS1_PTS2',
"9": 'Stromal2',
"10": 'PTS1_PTS2',
"11": 'CNT',
"12": 'ICA',
"13": 'CD_PC1',
"14": 'TAL1',
"15": 'TAL2',
"16": 'PTS3',
"17": 'Immune1',
"18": 'DCT_CNT2',
"19": 'EC3',
"20": 'TAL3',
"21": 'ICB',
"22": 'DTL',
"23": 'DCT_CNT1',
"24": 'PTS1_PTS2',
"25": 'Podo',
"26": 'CD_PC2',
"27": 'Immune2',
"28": 'Stromal1',
"29": 'EC2',
"30": 'PEC',
"31": 'ATL',
"32": 'EC1',
"33": 'PT_prolif',
"35": 'Stromal2',
"36": 'Immune3',
"37": 'MD',
"38": 'PTS1_PTS2',
"39": 'Uro_TP6',
"40": 'unknown_EC?',
"41": 'EC4',
"42": 'injPT',
"43": 'Immune_dublicates?',
"44": 'injPT',
"45": 'injPT',
"46": 'EC5',
"47": 'Stromal3',
"48": 'Stromal4',
"49": 'Stromal4',
}
adata.obs['annotation_Atlas_level2'] = adata.obs['annotation_Atlas_level2'].replace(Rename_cluster).astype('category')



In [ ]:
##remove clusters
clusters_remove=['34']

adata = adata[~adata.obs['leiden_scVI_2_0'].isin(clusters_remove),:]
adata

In [ ]:
sc.tl.leiden(adata, key_added="leiden_scVI_3_0", resolution=3.0, random_state=seed)

In [ ]:
sc.pl.umap(adata, color=['leiden_scVI_3_0'], legend_loc='on data')

In [ ]:
sc.set_figure_params(figsize=(40,10))
sc.pl.violin(adata, keys='nFeature_RNA', groupby='leiden_scVI_3_0')
sc.pl.violin(adata, keys='nCount_RNA', groupby='leiden_scVI_3_0')
sc.pl.violin(adata, keys='percent.mt', groupby='leiden_scVI_3_0')

In [ ]:
#set clustering of interest
adata.obs['leiden'] = adata.obs['leiden_scVI_3_0']
#get marker genes
de_df = model.differential_expression(
    groupby="leiden")
de_df.head()
markers = {}
cats = adata.obs.leiden.cat.categories
for i, c in enumerate(cats):
    cid = "{} vs Rest".format(c)
    leiden_scVI_df = de_df.loc[de_df.comparison == cid]
    leiden_scVI_df = leiden_scVI_df[leiden_scVI_df.lfc_mean > 0]
    leiden_scVI_df = leiden_scVI_df[leiden_scVI_df["bayes_factor"] > 3]
    leiden_scVI_df = leiden_scVI_df[leiden_scVI_df["non_zeros_proportion1"] > 0.1]
    leiden_scVI_df = leiden_scVI_df[leiden_scVI_df["lfc_mean"] > 0.75]
    markers[c] = leiden_scVI_df.index.tolist()[:3]
    sc.tl.dendrogram(adata, groupby="leiden", use_rep="X_scVI")
    sc.pl.dotplot(
    adata,
    markers,
    groupby='leiden',
    dendrogram=True,
    color_map="Blues",
    swap_axes=True,
    use_raw=True,
    standard_scale="var",
)

In [ ]:
##remove clusters
clusters_remove=['51']

adata = adata[~adata.obs['leiden_scVI_3_0'].isin(clusters_remove),:]
adata

In [ ]:
#we load subclustered objects and transfer the finer cell type labels

### load a subclustered object and transfer annotation 
#adata_subclustered = scvi.data.read_h5ad('Atlas6.1_PTcleaned_3000HVG_integrated_V3.h5ad')

#create new annotatiopn level - should be the level that was used for subsetting 
adata.obs["annotation_Atlas_level3"] = adata.obs.annotation_Atlas_level1

#change annotation for all subsetted cells to "delete" (not for PECs!)
Rename_cluster = {
"PTS1_PTS2": 'delete',
"injPT": 'delete',
"PTS3": 'delete',
"PT_prolif": 'delete',
"PEC": 'delete',
}
adata.obs['annotation_Atlas_level3'] = adata.obs['annotation_Atlas_level3'].replace(Rename_cluster).astype('category')

#check if worked 
sc.pl.umap(adata, color=['annotation_Atlas_level3'])

In [ ]:
#add higher leiden resolution cell type - PEC from level2

# Create a mask indicating which cells have leiden resolution 2 equal to ""
mask = adata.obs['annotation_Atlas_level2'] == "PEC"

# Add "" to the list of categories for the annotation column
adata.obs['annotation_Atlas_level3'].cat.add_categories(["PEC"], inplace=True)

# Update the annotation for cells with leiden resolution 2 
adata.obs.loc[mask, 'annotation_Atlas_level3'] = "PEC"

#check if worked 
sc.pl.umap(adata, color=['annotation_Atlas_level3'])

In [ ]:
#add cell type from sublustered adata - PTS1
adata.obs["annotation_transfer"] = adata_subclustered.obs['annotation_Atlas']

# Create a mask indicating which cells have leiden resolution 2 equal to ""
mask = adata.obs['annotation_transfer'] == "PTS1"

# Add "" to the list of categories for the annotation column
#adata.obs['annotation_Atlas_level3'].cat.add_categories(["PTS1"], inplace=True)

# Update the annotation for cells with leiden resolution 2 
adata.obs.loc[mask, 'annotation_Atlas_level3'] = "PTS1"

#check if worked 
sc.pl.umap(adata, color=['annotation_Atlas_level3'])

In [ ]:
#add cell type from sublustered adata - PTS2, PTS3, injPT, prolif_PT, PT_prolif_unknown_EBF1

# Create a mask indicating which cells have leiden resolution 2 equal to ""
mask = adata.obs['annotation_transfer'] == "PTS2"

# Add "" to the list of categories for the annotation column
adata.obs['annotation_Atlas_level3'].cat.add_categories(["PTS2", "PTS3", "injPT", "prolif_PT", "PT_prolif_unknown_EBF1"], inplace=True)

# Update the annotation for cells with leiden resolution 2 
adata.obs.loc[mask, 'annotation_Atlas_level3'] = "PTS2"



# Create a mask indicating which cells have leiden resolution 2 equal to ""
mask = adata.obs['annotation_transfer'] == "PTS3"

# Update the annotation for cells with leiden resolution 2 
adata.obs.loc[mask, 'annotation_Atlas_level3'] = "PTS3"




# Create a mask indicating which cells have leiden resolution 2 equal to ""
mask = adata.obs['annotation_transfer'] == "injPT"

# Update the annotation for cells with leiden resolution 2 
adata.obs.loc[mask, 'annotation_Atlas_level3'] = "injPT"




# Create a mask indicating which cells have leiden resolution 2 equal to ""
mask = adata.obs['annotation_transfer'] == "prolif_PT"

# Update the annotation for cells with leiden resolution 2 
adata.obs.loc[mask, 'annotation_Atlas_level3'] = "prolif_PT"




# Create a mask indicating which cells have leiden resolution 2 equal to ""
mask = adata.obs['annotation_transfer'] == "PT_prolif_unknown_EBF1"

# Update the annotation for cells with leiden resolution 2 
adata.obs.loc[mask, 'annotation_Atlas_level3'] = "PT_prolif_unknown_EBF1"



#check if worked 
sc.pl.umap(adata, color=['annotation_Atlas_level3'])

In [ ]:
#last step - remove "delete" = cells which were removed during subclustering 
clusters_remove=['delete']

adata = adata[~adata.obs['annotation_Atlas_level3'].isin(clusters_remove),:]
sc.pl.umap(adata, color=['annotation_Atlas_level3'])

In [ ]:
#add cell type from sublustered adata - PTS1
adata.obs["annotation_transfer"] = adata.obs['annotation_Atlas_level2']

# Create a mask indicating which cells have leiden resolution 2 equal to ""
mask = adata.obs['annotation_transfer'] == "MD"

# Add "" to the list of categories for the annotation column
adata.obs['annotation_Atlas_level3'].cat.add_categories(["MD"], inplace=True)

# Update the annotation for cells with leiden resolution 2 
adata.obs.loc[mask, 'annotation_Atlas_level3'] = "MD"

#check if worked 
sc.pl.umap(adata, color=['annotation_Atlas_level3'])

In [ ]:
#add cell type from sublustered adata - PTS1
adata.obs["annotation_transfer"] = adata.obs['annotation_Atlas_level2']

# Create a mask indicating which cells have leiden resolution 2 equal to ""
mask = adata.obs['annotation_transfer'] == "TAL1"

# Add "" to the list of categories for the annotation column
adata.obs['annotation_Atlas_level3'].cat.add_categories(["MTAL"], inplace=True)

# Update the annotation for cells with leiden resolution 2 
adata.obs.loc[mask, 'annotation_Atlas_level3'] = "MTAL"

#check if worked 
sc.pl.umap(adata, color=['annotation_Atlas_level3'])

In [ ]:
#add cell type from sublustered adata - PTS1
adata.obs["annotation_transfer"] = adata.obs['annotation_Atlas_level2']

TALS = ["TAL2", "TAL3", "TAL4"]

# Create a mask indicating which cells have leiden resolution 2 equal to ""
mask = adata.obs['annotation_transfer'].isin(TALS)

# Add "" to the list of categories for the annotation column
#adata.obs['annotation_Atlas_level3'].cat.add_categories(["CTAL"], inplace=True)

# Update the annotation for cells with leiden resolution 2 
adata.obs.loc[mask, 'annotation_Atlas_level3'] = "CTAL"

#check if worked 
sc.pl.umap(adata, color=['annotation_Atlas_level3'])

In [ ]:
#add cell type from sublustered adata - PTS1
adata.obs["annotation_transfer"] = adata.obs['annotation_Atlas_level2']

# Create a mask indicating which cells have leiden resolution 2 equal to ""
mask = adata.obs['annotation_transfer'] == "ATL"

# Add "" to the list of categories for the annotation column
adata.obs['annotation_Atlas_level3'].cat.add_categories(["ATL"], inplace=True)

# Update the annotation for cells with leiden resolution 2 
adata.obs.loc[mask, 'annotation_Atlas_level3'] = "ATL"


In [ ]:
#add cell type from sublustered adata - PTS1
adata.obs["annotation_transfer"] = adata.obs['annotation_Atlas_level2']

# Create a mask indicating which cells have leiden resolution 2 equal to ""
mask = adata.obs['annotation_transfer'] == "DTL"

# Add "" to the list of categories for the annotation column
adata.obs['annotation_Atlas_level3'].cat.add_categories(["DTL"], inplace=True)

# Update the annotation for cells with leiden resolution 2 
adata.obs.loc[mask, 'annotation_Atlas_level3'] = "DTL"

adata.obs['annotation_Atlas_level3'].cat.remove_categories('DTL_ATL', inplace=True)

#check if worked 
sc.pl.umap(adata, color=['annotation_Atlas_level3'])

In [ ]:
Rename_cluster = {
"TAL": 'delete',
}
adata.obs['annotation_Atlas_level3'] = adata.obs['annotation_Atlas_level3'].replace(Rename_cluster).astype('category')
#last step - remove "delete" = cells which were removed during subclustering 
clusters_remove=['delete']

adata = adata[~adata.obs['annotation_Atlas_level3'].isin(clusters_remove),:]
sc.pl.umap(adata, color=['annotation_Atlas_level3'])

In [ ]:
Rename_cluster = {
"unknown2": 'delete',
}
adata.obs['annotation_Atlas_level3'] = adata.obs['annotation_Atlas_level3'].replace(Rename_cluster).astype('category')
#last step - remove "delete" = cells which were removed during subclustering 
clusters_remove=['delete']

adata = adata[~adata.obs['annotation_Atlas_level3'].isin(clusters_remove),:]
sc.pl.umap(adata, color=['annotation_Atlas_level3'])

In [ ]:
#add cell type from sublustered adata - PTS1
adata.obs["annotation_transfer"] = adata.obs['leiden_scVI_3_0']

# Create a mask indicating which cells have leiden resolution 2 equal to ""
mask = adata.obs['annotation_transfer'] == "44"

# Add "" to the list of categories for the annotation column
adata.obs['annotation_Atlas_level3'].cat.add_categories(["ICA2"], inplace=True)

# Update the annotation for cells with leiden resolution 2 
adata.obs.loc[mask, 'annotation_Atlas_level3'] = "ICA2"

sc.pl.umap(adata, color=['annotation_Atlas_level3'])

In [ ]:
adata.write('Atlas6.2_3000HVG_integrated_V2.h5ad')